In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
os.symlink('/content/drive/MyDrive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [3]:
# Seed 고정
import random
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [4]:
import pandas as pd
from glob import glob

os.chdir("/content/drive/MyDrive/DACON_Image/")

train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))

train_y = pd.read_csv("train.csv")

train_y['label'][train_y['label'] == '10-1'] = 10 ## label : 10-1 -> 10
train_y['label'][train_y['label'] == '10-2'] = 0 ## Label : 10-2 -> 0
train_y['label'] = train_y['label'].apply(lambda x : int(x)) ## Dtype : object -> int

train_labels = train_y['label']

label_unique = sorted(np.unique(train_labels)) # np.unique 는 고유값 정렬해준다.
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [5]:
import cv2
import gc

def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (256, 256))

    gc.collect()
    torch.cuda.empty_cache()
    
    return img

In [6]:
from tqdm.auto import tqdm
# tqbm은 진행률을 표기해주는 모듈, 리스트 등 이터러블 돌릴 수 있다.
# tqbm parameter = iterable : 반복자 객체, desc : 텍스트, total : 반복량

train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

  0%|          | 0/858 [00:00<?, ?it/s]

  0%|          | 0/215 [00:00<?, ?it/s]

In [7]:
from albumentations.augmentations.transforms import Normalize
import albumentations as A
from albumentations.pytorch import ToTensorV2
import ttach as tta

train_transform = A.Compose([
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
        A.RandomGamma(gamma_limit=(90, 110)),
        A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10),
        A.Transpose(),
        A.RandomRotate90(),
        A.OneOf([A.NoOp(), A.MultiplicativeNoise(), A.GaussNoise(), A.ISONoise()]),
        A.OneOf(
            [
                A.NoOp(p=0.8),
                A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10),
                A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10)
            ],
            p=0.2,
        ),
        A.OneOf([A.ElasticTransform(), A.GridDistortion(), A.NoOp()]),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
])

test_transform = A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
])

In [15]:
import torchvision.datasets as datasets # 이미지 데이터셋 집합체
import torchvision.transforms as transforms # 이미지 변환 툴
import torch.nn as nn

from torchvision import models

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score, accuracy_score
import time

class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            img = train_transform(image=img)
        
        if self.mode=='test':
            img = test_transform(image=img)
        
        label = self.labels[idx]
        return img, label

class Network_b3(nn.Module):
    def __init__(self):
        super(Network_b3, self).__init__()
        self.model = timm.create_model('efficientnet_b6', pretrained=False, num_classes=11)

    def forward(self, x):
        x = self.model(x)

        return x

class Network_wrn(nn.Module):
    def __init__(self):
        super(Network_wrn, self).__init__()
        self.model = timm.create_model('wide_resnet50_2', pretrained=False, num_classes=11)
        
    def forward(self, x):
        x = self.model(x)

        return x

In [9]:
# Get Dataloader
batch_size = 32
epochs = 120

#CustomDataset class를 통하여 train dataset생성
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')

#만든 train dataset를 DataLoader에 넣어 batch 만들기
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=0)

#CustomDataset class를 통하여 train dataset생성
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')

#만든 train dataset를 DataLoader에 넣어 batch 만들기
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, num_workers=0)

# #vaildation 에서도 적용
# vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
# vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [10]:
import torch.nn as nn # 신경망들이 포함됨
import torch.nn.init as init # 텐서에 초기값을 줌
import timm

In [11]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [12]:
model_wrn = Network_wrn().to(device)

optimizer = torch.optim.Adam(model_wrn.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

In [13]:
path = './weights/'
if not os.path.isdir(path):
    os.mkdir(path)

for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model_wrn.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0]['image'], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model_wrn(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()

    train_f1 = score_function(train_y, train_pred)
    
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

    # 모델 저장
    torch.save({
                'epoch': epoch,
                'model_state_dict': model_wrn.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                "scaler": scaler.state_dict(),
                'loss': loss,
                }, f"{path}/wrn_model.pt")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 1/120    time : 32s/3794s
TRAIN    loss : 2.43432    f1 : 0.09887


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 2/120    time : 26s/3012s
TRAIN    loss : 2.31814    f1 : 0.13646


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


...

epoch : 118/120    time : 25s/50s
TRAIN    loss : 0.31143    f1 : 0.89581


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 119/120    time : 27s/27s
TRAIN    loss : 0.33934    f1 : 0.89892


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 120/120    time : 26s/0s
TRAIN    loss : 0.28448    f1 : 0.91307


In [16]:

model_b3 = Network_b3().to(device)

optimizer = torch.optim.Adam(model_b3.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

In [17]:
path = './weights/'
if not os.path.isdir(path):
    os.mkdir(path)

for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model_b3.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0]['image'], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model_b3(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()

    train_f1 = score_function(train_y, train_pred)
    
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

    # 모델 저장
    torch.save({
                'epoch': epoch,
                'model_state_dict': model_b3.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                "scaler": scaler.state_dict(),
                'loss': loss,
                }, f"{path}/b3_model.pt")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 1/120    time : 41s/4879s
TRAIN    loss : 5.16876    f1 : 0.06505


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 2/120    time : 32s/3732s
TRAIN    loss : 4.12724    f1 : 0.08027


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


...

epoch : 118/120    time : 33s/67s
TRAIN    loss : 0.63293    f1 : 0.79201


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 119/120    time : 34s/34s
TRAIN    loss : 0.62539    f1 : 0.78280


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


epoch : 120/120    time : 33s/0s
TRAIN    loss : 0.57494    f1 : 0.80018


In [18]:
# efficientnet_b3 모델 불러오기

loaded_model = torch.load('/content/drive/MyDrive/DACON_Image/weights/b3_model.pt')
model_b3 = Network_b3().to(device)
model_b3.load_state_dict(loaded_model['model_state_dict'])

# wide_resnet50_2 모델 불러오기

loaded_model = torch.load('/content/drive/MyDrive/DACON_Image/weights/wrn_model.pt')
model_wrn = Network_wrn().to(device)
model_wrn.load_state_dict(loaded_model['model_state_dict'])

<All keys matched successfully>

In [19]:
tta_transforms = tta.Compose(
    [
        tta.Rotate90(angles=[0, 90, 180, 270]),
        tta.Multiply([0.9, 1])
    ]
)

tta_model_b3 = tta.ClassificationTTAWrapper(model_b3, tta_transforms)
tta_model_wrn = tta.ClassificationTTAWrapper(model_wrn, tta_transforms)

In [20]:
tta_model_b3.eval()
tta_model_wrn.eval()
f_pred = []

with torch.no_grad  ():
    for batch in (test_loader):
        x = torch.tensor(batch[0]['image'], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            # ensemble
            pred_b3 = tta_model_b3(x)
            pred_wrn = tta_model_wrn(x)
            # print(pred_wrn)
            pred = pred_b3 + pred_wrn
            # print(pred)
            f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


In [21]:
#print(f_pred)

In [22]:
# label_decoder = {val:key for key, val in label_unique.items()}
# f_result = [label_decoder[result] for result in f_pred]

submission = pd.read_csv('/content/drive/MyDrive/DACON_Image/sample_submission.csv')
submission['label'] = f_pred

In [23]:
submission['label'][submission['label'] == 10] = '10-1' ## label : 10 -> '10-1'
submission['label'][submission['label'] == 0] = '10-2' ## Label : 0 -> '10-2'
submission['label'] = submission['label'].apply(lambda x : str(x)) ## Dtype : int -> object

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
submission.head()
# print(f_result)

,file_name,label
0,001.png,1
1,002.png,8
2,003.png,1
3,004.png,6
4,005.png,8


In [25]:
submission.to_csv('submit.csv', index=False)

In [26]:
submission.to_csv('/content/drive/MyDrive/DACON_Image/', index=False)

IsADirectoryError: ignored